In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils import load_dataset
from rnn_utils import *

%load_ext autoreload
%autoreload 2

In [2]:
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y = load_dataset()

# fetch number of examples and picture size
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

# reshape the input dataset as (nx, m)
train_set_x_flatten = train_set_x_orig.reshape(m_train, -1).T
test_set_x_flatten = test_set_x_orig.reshape(m_test, -1).T

# normalize
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

Reading train-images-idx3-ubyte with 60000 images of shape: (28, 28)
Reading train-labels-idx1-ubyte with shape: (60000,)
Reading t10k-images-idx3-ubyte with 10000 images of shape: (28, 28)
Reading t10k-labels-idx1-ubyte with shape: (10000,)


In [3]:
# Clip gradients
def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    
    dWaa = gradients["dWaa"] 
    dWax = gradients["dWax"]
    dWya = gradients["dWya"]
    db = gradients["db"]
    dby = gradients["dby"]

    for gradient in [dWaa, dWax, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
        
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [76]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.1):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    
    ### START CODE HERE ###
    
    # Forward propagate through time (≈1 line)
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time (≈1 line)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    #gradients = clip(gradients, 5)
    
    # Update parameters (≈1 line)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    ### END CODE HERE ###
    
    return loss, gradients

In [97]:
# GRADED FUNCTION: model

def model(train_set_x, train_set_y, n_a = 800, n_x = 784):
    """
    Trains the model for handwritten digit recognition. 
    
    Arguments:
    data -- text corpus
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text, size of the vocabulary
    
    Returns:
    parameters -- learned parameters
    """

    # Retrieve n_x and n_y 
    n_y = 10
    #m = train_set_x.shape[1]
    m = 10000
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss, don't worry about it)
    #loss = get_initial_loss(vocab_size, dino_names)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))

    for j in range(m):
        
        X = train_set_x[:, j]
        Y = train_set_y[:, j]
        #print(Y)
        
        curr_loss, gradients = optimize(X, Y, a_prev, parameters, 0.9)
        
        #loss = smooth(loss, curr_loss)
        
        if (j % 1000 == 0):
            print('Iteration: %d, Loss: %f' % (j, curr_loss) + '\n')

    return parameters

In [98]:
parameters = model(train_set_x, train_set_y)

Iteration: 0, Loss: 2.302585

Iteration: 1000, Loss: 2.239024

Iteration: 2000, Loss: 1.257213

Iteration: 3000, Loss: 2.174311

Iteration: 4000, Loss: 3.153352

Iteration: 5000, Loss: 2.022073

Iteration: 6000, Loss: 1.177417

Iteration: 7000, Loss: 2.600877

Iteration: 8000, Loss: 2.758445

Iteration: 9000, Loss: 2.487665



In [99]:
def predict(X, Y, parameters):
    
    Waa, Wax, Wya, b, by = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['b'], parameters['by']
    n_a = Waa.shape[0]
    
    a0 = np.zeros((n_a, 1))
    
    _, cache = rnn_forward(X, Y, a0, parameters, width=28, height=28)
    
    y_hat, _, _ = cache
    
    return y_hat

In [106]:
X_pred = train_set_x[:, 100]
Y_pred = train_set_y[:, 100]
y_hat_pred = predict(X_pred, Y_pred, parameters)

In [107]:
y_hat_pred

array([[0.12506605],
       [0.07163221],
       [0.07134264],
       [0.07361245],
       [0.10070557],
       [0.05484342],
       [0.07003069],
       [0.25459952],
       [0.09744796],
       [0.08071948]])

In [105]:
np.argmax(y_hat_pred)

7

In [ ]:
Wya = parameters['Wya']

In [ ]:
Wya[9, :]